# COMP 3106 Project Code 

## Authors 
Kyle Knobloch, Balreet Kaur Dhillon, Yuxiao Chen 

## Data

Waste Water Data download: https://github.com/Big-Life-Lab/PHESD

Ottawa Testin data: https://www.arcgis.com/home/item.html?id=26c902bf1da44d3d90b099392b544b81 

## Notes
I added stuff from the guide (https://realpython.com/python-ai-neural-network/#python-ai-starting-to-build-your-first-neural-network). 

Looks like there's a class that we could rely on heavily to do this work. - KYLE 

## TODO
- [X] Data Compile 
- [ ] Get it to work with NeuralNetwork class from tutorial
- [ ] Get it to work with the sklearn (proof of working) 
- [ ] Presentation with our findings 

In [ ]:
import numpy as np
import math
import requests
import matplotlib as plot
from io import StringIO

waster_water_url = 'https://raw.githubusercontent.com/Big-Life-Lab/PHESD/main/Wastewater/Ottawa/Data/wastewater_virus.csv'
ww_r = requests.get(waster_water_url).text
wastewater = np.genfromtxt(StringIO(ww_r), delimiter=",", dtype=None, encoding=None)
#print(wastewater[:5])


covid_testing_url = 'https://www.arcgis.com/sharing/rest/content/items/26c902bf1da44d3d90b099392b544b81/data'
covid_r = requests.get(covid_testing_url).text
covid_r = covid_r.replace('"Date","Number of Tests, Excluding LTCH","Daily % Positivity, Excluding LTCH","Number of Tests in LTCH","LTCH Daily % Positivity"', 
                          '"Date","Number of Tests Excluding LTCH","Daily % Positivity Excluding LTCH","Number of Tests in LTCH","LTCH Daily % Positivity"')
covidtests = np.genfromtxt(StringIO(covid_r), delimiter=",", dtype=None, encoding=None)
#print(covidtests[:5])

covid_totals = []
for i in covidtests:
  if i[0] == '"Date"':
    covid_totals.append(["Total positive"])
  else:
    covid_totals.append([int((float(i[1]) * float(i[2]) * 0.01) + (float(i[3]) * float(i[4]) * 0.01))])

covidtests = np.append(covidtests, covid_totals, axis=1)
#print(covidtests[:10])


ww_covid = []
# WITH DATE 
#ww_covid.append(["date", "wastewater level", "covid count positive on that day"])
# NO DATE
ww_covid.append(["wastewater level", "covid count positive on that day"])

ww_count = 1 # ww count
co_count = 1 # covid count 
while True: 
  if co_count >= len(covidtests) or ww_count >= len(wastewater):
    break

  # covid testing started before ww testing
  #print(covidtests[co_count][0].replace('"','') + " " + wastewater[ww_count][0])
  if covidtests[co_count][0].replace('"','') == wastewater[ww_count][0]:
    # WITH DATE
    #ww_covid.append([wastewater[ww_count][0], float(wastewater[ww_count][5]) + float(wastewater[ww_count][7]), float(covidtests[co_count][5])])
    # NO DATE
    ww_covid.append([float(wastewater[ww_count][5]) + float(wastewater[ww_count][7]), float(covidtests[co_count][5])])

    ww_count = ww_count + 1
  # Always increment 
  co_count = co_count + 1

##########################
### DATA #################
#####################
ww_covid = np.array(ww_covid)
print(ww_covid[:15])
print("Number of Records: " + str(len(ww_covid)))


# covN1: SARS-CoV-2 nucleocapsid gene N1 normalized with nPMMoV: Pepper mild mottle virus
# covN2: SARS-CoV-2 nucleocapsid gene N2 normalized with nPMMoV: Pepper mild mottle virus
# Added together gets the waste water value
# https://github.com/Big-Life-Lab/PHES-ODM/blob/main/metadata_en.md



[['wastewater level' 'covid count positive on that day']
 ['0.00038713500000000004' '18.0']
 ['0.000173621' '31.0']
 ['0.000138118' '31.0']
 ['5.9589e-05' '21.0']
 ['9.7456e-05' '9.0']
 ['3.9861e-05' '2.0']
 ['4.9405000000000006e-05' '0.0']
 ['5.9341e-05' '10.0']
 ['4.0598999999999995e-05' '2.0']
 ['7.487100000000001e-05' '4.0']
 ['0.00032656899999999997' '1.0']
 ['0.00010175' '3.0']
 ['0.00010588700000000001' '1.0']
 ['0.000101206' '6.0']]
Number of Records: 606


In [ ]:
input_vector = [1.72, 1.23]
weights_1 = [1.26, 0]
weights_2 = [2.17, 0.33]

first_indexes_mult = input_vector[0] * weights_1[0]
second_indexes_mult = input_vector[1] * weights_1[1]
dot_product_1 = first_indexes_mult + second_indexes_mult

print(f"The dot product is: {dot_product_1}")

The dot product is: 2.1672


In [ ]:
dot_product_1 = np.dot(input_vector, weights_1)
print(f"The dot product is: {dot_product_1}")

The dot product is: 2.1672


In [ ]:
dot_product_2 = np.dot(input_vector, weights_2)
print(f"The dot product 2 is: {dot_product_2}")

The dot product 2 is: 4.1383


In [ ]:
input_vector = np.array([1.66, 1.56])
weights_1 = np.array([1.45, -0.66])
bias = np.array([0.0])
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def make_prediction(input_vector, weights, bias):
  layer_1 = np.dot(input_vector, weights) + bias
  layer_2 = sigmoid(layer_1)
  return layer_2

prediction = make_prediction(input_vector, weights_1, bias)
print(f"The prediction result is: {prediction}")

The prediction result is: [0.7985731]


In [ ]:
input_vector = np.array([2, 1.5])
prediction = make_prediction(input_vector, weights_1, bias)
print(f"The prediction result is: {prediction}")

The prediction result is: [0.87101915]


In [ ]:
target = 0

mse = np.square(prediction - target)
print(f"Prediction: {prediction}; Error: {mse}")

Prediction: [0.87101915]; Error: [0.75867436]


In [ ]:
derivative = 2 * (prediction - target)
print(f"The derivative is {derivative}")

The derivative is [1.7420383]


In [ ]:
 weights_1 = weights_1 - derivative
 prediction = make_prediction(input_vector, weights_1, bias)
 error = (prediction - target) ** 2
 print(f"Prediction: {prediction}; Error: {error}")

Prediction: [0.01496248]; Error: [0.00022388]


In [ ]:
# CLASS
class NeuralNetwork:
    def __init__(self, learning_rate):
        self.weights = np.array([np.random.randn(), np.random.randn()])
        self.bias = np.random.randn()
        self.learning_rate = learning_rate

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _sigmoid_deriv(self, x):
        return self._sigmoid(x) * (1 - self._sigmoid(x))

    def predict(self, input_vector):
        layer_1 = np.dot(input_vector, self.weights) + self.bias
        layer_2 = self._sigmoid(layer_1)
        prediction = layer_2
        return prediction

    def _compute_gradients(self, input_vector, target):
        layer_1 = np.dot(input_vector, self.weights) + self.bias
        layer_2 = self._sigmoid(layer_1)
        prediction = layer_2

        derror_dprediction = 2 * (prediction - target)
        dprediction_dlayer1 = self._sigmoid_deriv(layer_1)
        dlayer1_dbias = 1
        dlayer1_dweights = (0 * self.weights) + (1 * input_vector)

        derror_dbias = (
            derror_dprediction * dprediction_dlayer1 * dlayer1_dbias
        )
        derror_dweights = (
            derror_dprediction * dprediction_dlayer1 * dlayer1_dweights
        )

        return derror_dbias, derror_dweights

    def _update_parameters(self, derror_dbias, derror_dweights):
        self.bias = self.bias - (derror_dbias * self.learning_rate)
        self.weights = self.weights - (
            derror_dweights * self.learning_rate
        )
    
    def train(self, input_vectors, targets, iterations):
        cumulative_errors = []
        for current_iteration in range(iterations):
            # Pick a data instance at random
            random_data_index = np.random.randint(len(input_vectors))

            input_vector = input_vectors[random_data_index]
            target = targets[random_data_index]

            # Compute the gradients and update the weights
            derror_dbias, derror_dweights = self._compute_gradients(
                input_vector, target
            )

            self._update_parameters(derror_dbias, derror_dweights)

            # Measure the cumulative error for all the instances
            if current_iteration % 100 == 0:
                cumulative_error = 0
                # Loop through all the instances to measure the error
                for data_instance_index in range(len(input_vectors)):
                    data_point = input_vectors[data_instance_index]
                    target = targets[data_instance_index]

                    prediction = self.predict(data_point)
                    error = np.square(prediction - target)

                    cumulative_error = cumulative_error + error
                cumulative_errors.append(cumulative_error)

        return cumulative_errors

# MAIN
learning_rate = 0.1

neural_network = NeuralNetwork(learning_rate)

neural_network.predict(input_vector)

0.00042518255387658985

In [ ]:
from sklearn.model_selection import train_test_split
x = np.arange(1, 25).reshape(12, 2)
y = np.array([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=4, random_state=4)
x_train

array([[17, 18],
       [ 5,  6],
       [23, 24],
       [ 1,  2],
       [ 3,  4],
       [11, 12],
       [15, 16],
       [21, 22]])